<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=9118e2da35c8a027402e7c8581d3b185daab25d093f090d47f5179592b119f84
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
!wget 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip'

--2022-06-12 23:14:43--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  27.2MB/s    in 8.7s    

2022-06-12 23:14:52 (28.7 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In [ ]:
!unzip /content/concrete_data_week4.zip

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [41]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [11]:
#Preprocessing
vgg_image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
#Train loader
vgg_train_loader = vgg_image_generator.flow_from_directory(
    '/content/concrete_data_week4/train',
    batch_size=100,
    target_size=(224,224),
    shuffle=True
)
#Valid loader
vgg_valid_loader = vgg_image_generator.flow_from_directory(
    '/content/concrete_data_week4/valid',
    batch_size=100,
    target_size=(224,224),
    shuffle=True
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [12]:
def load_pretrained_model():
  input_t = Input(shape=(224,224,3))
  model = VGG16(weights='imagenet',input_tensor=input_t,include_top=False)
  model.trainable = False
  return model

base_model = load_pretrained_model()

In [13]:
def add_head(feature_extractor_model):
  prev_output  = feature_extractor_model.output
  head_flatten = Flatten()(prev_output)
  head_dense1  = Dense(64,activation='relu',kernel_regularizer='l2')(head_flatten)
  head_drop    = Dropout(0.5)(head_dense1)
  head_dense2  = Dense(2,activation='softmax')(head_drop) 

  model = Model(inputs=feature_extractor_model.input,outputs=head_dense2)

  opt = Adam(1e-4)
  model.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )
  return model

In [14]:
model = add_head(base_model)
#model.summary()

In [15]:
def scheduler(epoch,lr):
  if epoch<1:
    return lr
  else:
    return lr*tf.math.exp(-0.105)
batch = 100
def train_model_gen(model,train_gen,val_gen,epochs):
  early_stop = EarlyStopping(patience=1,monitor='accuracy')

  
  return model.fit(
      train_gen,
      steps_per_epoch  = train_gen.samples // batch,
      validation_data  = val_gen,
      validation_steps = val_gen.samples // batch,
      epochs=epochs,
      callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),early_stop],
      verbose=1
      )

In [16]:
history = train_model_gen(model,vgg_train_loader,vgg_valid_loader,3)

Epoch 1/3
300/300 [==============================] - 148s 445ms/step - loss: 0.6524 - accuracy: 0.9879 - val_loss: 0.3897 - val_accuracy: 0.9962 - lr: 1.0000e-04
Epoch 2/3
300/300 [==============================] - 117s 388ms/step - loss: 0.3103 - accuracy: 0.9981 - val_loss: 0.2470 - val_accuracy: 0.9980 - lr: 9.0032e-05
Epoch 3/3
300/300 [==============================] - 116s 385ms/step - loss: 0.2061 - accuracy: 0.9988 - val_loss: 0.1746 - val_accuracy: 0.9971 - lr: 8.1058e-05


In [ ]:
history.history

{'accuracy': [0.9841811060905457, 0.997491717338562],
 'loss': [0.6830583214759827, 0.32709500193595886],
 'lr': [1e-04, 9.003245e-05],
 'val_accuracy': [0.9964210391044617, 0.996842086315155],
 'val_loss': [0.4073544144630432, 0.2653428912162781]}

In [17]:
def plots(history,model_name:str):
  plt.figure(figsize=(20,10))
  plt.subplot(121)
  try:
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
  except KeyError:
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
  plt.title(f'Accuracy vs epochs {model_name}')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend(['training', 'validation'])
  plt.subplot(122)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.legend(['training_loss','val_loss'])
  plt.show()

In [ ]:
plots(history,'vgg16')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [19]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

In [20]:
#Preprocessing
resnet_image_generator = ImageDataGenerator(preprocessing_function=resnet_preprocess)
#Train loader
resnet_train_loader = resnet_image_generator.flow_from_directory(
    '/content/concrete_data_week4/train',
    batch_size=100,
    target_size=(224,224),
    shuffle=True
)
#Valid loader
resnet_valid_loader = resnet_image_generator.flow_from_directory(
    '/content/concrete_data_week4/valid',
    batch_size=100,
    target_size=(224,224),
    shuffle=True
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [21]:
def load_resnet_pretrained_model():
  input_t = Input(shape=(224,224,3))
  model = ResNet50(weights='imagenet',input_tensor=input_t,include_top=False)
  model.trainable = False
  return model

resnet_base_model = load_resnet_pretrained_model()

94781440/94765736 [==============================] - 1s 0us/step


In [22]:
resnet_model = add_head(resnet_base_model)
#resnet_model.summary()

In [23]:
resnet_history = train_model_gen(resnet_model,resnet_train_loader,resnet_valid_loader,2)

Epoch 1/2
300/300 [==============================] - 114s 368ms/step - loss: 0.4449 - accuracy: 0.9948 - val_loss: 0.2043 - val_accuracy: 0.9980 - lr: 1.0000e-04
Epoch 2/2
300/300 [==============================] - 109s 362ms/step - loss: 0.1512 - accuracy: 0.9980 - val_loss: 0.1167 - val_accuracy: 0.9975 - lr: 9.0032e-05


In [ ]:
plots(resnet_history,'resnet50')

In [25]:
resnet_test_loader = resnet_image_generator.flow_from_directory(
    '/content/concrete_data_week4/test',
    batch_size=100,
    target_size=(224,224),
    shuffle=False
)
vgg_test_loader = vgg_image_generator.flow_from_directory(
    '/content/concrete_data_week4/test',
    batch_size=100,
    target_size=(224,224),
    shuffle=False
)

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [28]:
#vgg16
vgg_loss, vgg_acc = model.evaluate(
    vgg_test_loader,
)

5/5 [==============================] - 2s 289ms/step - loss: 0.1673 - accuracy: 1.0000


In [29]:
#Resnet50
resnet_los, resnet_acc = resnet_model.evaluate(
    resnet_test_loader
)

5/5 [==============================] - 2s 272ms/step - loss: 0.1087 - accuracy: 1.0000


In [33]:
print("Results of both models on test data\n")
print('*ResNet50-- loss: {:.2f}, accuracy: {}'.format(resnet_los,resnet_acc))
print('*VGG16----- loss: {:.2f}, accuracy: {}'.format(vgg_loss,vgg_acc))

Results of both models on test data

*ResNet50-- loss: 0.11, accuracy: 1.0
*VGG16----- loss: 0.17, accuracy: 1.0


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [36]:
resnet_pred = resnet_model.predict(
    resnet_test_loader,
    steps=1
)

vgg_pred = model.predict(
    vgg_test_loader,
    steps=1
)

In [52]:
print('The results for the first 5 elements on test data with resNet50 model are: ',
['Negative' if np.argmax(x)==0 else 'Positive' for x in resnet_pred[0:5,:]]
)

The results for the first 5 elements on test data with resNet50 model are:  ['Negative', 'Negative', 'Negative', 'Negative', 'Negative']


In [51]:
print('The results for the first 5 elements on test data with VGG16 model are: ',
['Negative' if np.argmax(x)==0 else 'Positive' for x in vgg_pred[0:5,:]]
)

The results for the first 5 elements on test data with VGG16 model are:  ['Negative', 'Negative', 'Negative', 'Negative', 'Negative']


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).